<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_answer_KCBERT_nsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질문에 대한 답변을 선별하는 코드(BERT Next Sentence Prediction)(로컬)

In [ ]:
!pip install soynlp
!pip install datasets
!pip install accelerate -U
import accelerate
import random
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from datasets import Dataset, load_dataset, ClassLabel
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments, BertForMaskedLM, Trainer, TrainerCallback
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 k

#모델 학습

MLM으로 비지도 학습

In [ ]:
#훈련 데이터 불러오기
pretrain_dataset_path = '/content/drive/My Drive/KcBERT_pretrain_dataset.csv'
df_pretrain = pd.read_csv(pretrain_dataset_path, index_col = 0)
print(df_pretrain)

                                                    text
0                               저도 나중에 시간되면 자세히 공유하겠습니다.
1                                           새해 복 많이 받으세요
2      이전버전으로 가서 디플로이를 하려면 live버전과 sync를 맞추라고 하는 알림이 ...
3      아직은 초기 버전이기 때문에 일반적인 이커머스 제품에 커뮤니티 방 기능 밖에 추가되...
4                                         이 방법 시도해보셨을까요?
...                                                  ...
29180                              노코드 빌딩의 정수를 경험하셨네요 ㅎㅎ
29181                                         답변 감사드립니다.
29182                                           응원하겠습니다.
29183  그리고 헤더그룹의 Horizontal Alignment를 centered로 두시면 ...
29184                       저의 마지막댓글에 답변 부탁드려도 괜찮을까요 ㅜㅜ?

[29185 rows x 1 columns]


In [ ]:
#Huggingface의 원본 모델 로드
#BertForMaskedLM 구조에 맞게 모델 불러오기
HUGGINGFACE_MODEL_PATH = 'beomi/kcbert-base'

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = BertForMaskedLM.from_pretrained(HUGGINGFACE_MODEL_PATH)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Trainer가 알아서 gpu를 감지하고 모델과 데이터를 gpu로 옮겨줌
# #GPU 사용 코드
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# #파이프라인을 사용하는 경우 GPU로 모델을 옮겨준 뒤 파이프라인에 모델을 넣어줘야 함
# model.to(device)

In [ ]:
#soynlp tokenizer를 Pretrain 데이터로 학습
pretrain_list = df_pretrain['text'].to_list()

word_extractor = WordExtractor(min_frequency = 5)
word_extractor.train(pretrain_list)
word_score_table = word_extractor.extract()

training was done. used memory 1.603 Gb
all cohesion probabilities was computed. # words = 26581
all branching entropies was computed # words = 59896
all accessor variety was computed # words = 59896


In [ ]:
print(word_score_table['picker'])

Scores(cohesion_forward=0.4846039848864778, cohesion_backward=0.35785416966332295, left_branching_entropy=0, right_branching_entropy=1.1919038879225412, left_accessor_variety=0, right_accessor_variety=4, leftside_frequency=24, rightside_frequency=5)


In [ ]:
# 예제로 몇 가지 단어 점수 확인
for word, score in word_score_table.items():
    print(f"{word}: {score.cohesion_forward:.2f}")

In [ ]:
cohesion_score = {word:score.cohesion_forward for word, score in word_score_table.items()}

In [ ]:
Ltokenizer = LTokenizer(scores = cohesion_score)
Maxtokenizer = MaxScoreTokenizer(scores = cohesion_score)

In [ ]:
sentence = '자바스크립트 개발 경험이 있으신 경우라면 DateTime Picker 엘레먼트에 ID를 부여해서 자바스크립트 코드로 date 함수와 DOM을 핸들링해서 직접 처리해서 쉽게 구현하실 수가 있을 듯 합니다.'

print(Ltokenizer.tokenize(sentence))
print(Maxtokenizer.tokenize(sentence))

['자바스크립트', '개발', '경험이', '있으신', '경우', '라면', 'Dat', 'eTime', 'Picker', '엘레먼트', '에', 'ID', '를', '부여해서', '자바스크립트', '코드', '로', 'dat', 'e', '함수', '와', 'DOM을', '핸들링', '해서', '직접', '처리', '해서', '쉽게', '구현', '하실', '수가', '있을', '듯', '합니다', '.']
['자바스크립트', '개발', '경험이', '있으신', '경우', '라면', 'Dat', 'e', 'Time', 'Pic', 'ke', 'r', '엘레먼트', '에', 'ID', '를', '부여해서', '자바스크립트', '코드', '로', 'dat', 'e', '함수', '와', 'DOM을', '핸들링', '해서', '직접', '처리', '해서', '쉽게', '구현', '하실', '수가', '있을', '듯', '합니다', '.']


In [ ]:
sentence = ' 지원하는 기능이 많을수록 느립니다. BDK는 확실히 로딩속도가 느립니다.Natively는 기능이 적은 대신에 로딩속도는 조금 빠른 편이더군요. :)'

print(Ltokenizer.tokenize(sentence))
print(Maxtokenizer.tokenize(sentence))

['지원하는', '기능', '이', '많을수록', '느립니다.', 'BDK', '는', '확실히', '로딩속도가', '느립니다.', 'Natively는', '기능', '이', '적은', '대신에', '로딩속도', '는', '조금', '빠른', '편이', '더군요.', ':)']
['지', '원하는', '기능', '이', '많을수록', '느립니다.', 'BDK', '는', '확실히', '로딩', '속도', '가', '느립니다.', 'Natively', '는', '기능', '이', '적은', '대신에', '로딩', '속도', '는', '조금', '빠른', '편이', '더군요.', ':)']


In [ ]:
# 높은 cohesion 점수를 가진 단어들을 선택하여 단어사전 구성
threshold = 0.1  # 점수 기준 설정
new_vocab = [word for word, score in word_score_table.items() if score.cohesion_forward > threshold]

# 선택된 단어 출력
print(new_vocab)

In [ ]:
# 새로운 단어 토크나이저에 추가
num_added_toks = tokenizer.add_tokens(new_vocab)
print(f"Added {num_added_toks} tokens")

# 모델의 임베딩 크기 조정
model.resize_token_embeddings(len(tokenizer))

Added 12400 tokens


Embedding(40596, 768)

In [ ]:
#MLM 데이터셋 준비
corpus = pretrain_list
tokenized_corpus = tokenizer(corpus, add_special_tokens = True, max_length = model.config.max_position_embeddings, truncation = True, padding = 'max_length', return_tensors = 'pt')

In [ ]:
print(tokenized_corpus)

{'input_ids': tensor([[    2, 10306,  9206,  ...,     0,     0,     0],
        [    2, 18702,  1572,  ...,     0,     0,     0],
        [    2, 39402,  8606,  ...,     0,     0,     0],
        ...,
        [    2, 39950,    17,  ...,     0,     0,     0],
        [    2,  8108, 35185,  ...,     0,     0,     0],
        [    2, 29952, 10220,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
#MLM을 위한 데이터셋의 일부를 [MASK] 토큰으로 교체하는 함수
def mask_input_ids(input_ids, tokenizer, mlm_probability=0.15):
    labels = input_ids.clone()

    #각 입력에서 특수 토큰은 마스킹하지 않도록 설정
    special_tokens_mask = [
      torch.tensor(tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True), dtype = torch.bool)
      for val in input_ids]
    special_tokens_mask = torch.stack(special_tokens_mask)

    #마스킹 확률을 적용해 마스크 생성
    probability_matrix = torch.full(labels.shape, mlm_probability)
    probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()

    # PyTorch에서 사용하는 masked loss 계산을 위해 마스킹 되지 않은 부분은 -100으로 설정
    labels[~masked_indices] = -100

    # `[MASK]` 토큰으로 대체
    input_ids[masked_indices] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
    return input_ids, labels

In [ ]:
#과적합 방지를 위한 EarlyStopping 클래스
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=3):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def on_evaluate(self, args, state, control, **kwargs):
        # 현재 검증 손실 가져오기
        current_loss = kwargs['metrics']['eval_loss']

        # 최고 성능 갱신 또는 카운터 증가
        if self.best_score is None:
            self.best_score = current_loss
        elif current_loss > self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                control.should_training_stop = True
        else:
            self.best_score = current_loss
            self.counter = 0

In [ ]:
#MLM을 위한 masked 데이터셋 생성(MLM에서는 stratify 적용하지 않음.)
masked_inputs, labels = mask_input_ids(tokenized_corpus['input_ids'], tokenizer)
pretrain_dict = {'input_ids' : masked_inputs, 'attention_mask' : tokenized_corpus['attention_mask'], 'labels' : labels}
pretrain_dataset = Dataset.from_dict(pretrain_dict)
pretrain_dataset = dataset.train_test_split(test_size = 0.1)

In [ ]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_strategy = "steps",
    save_steps = 500,
    num_train_epochs = 3,
    save_total_limit = 2,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    warmup_steps = 300, #고려사항
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = pretrain_dataset['train'],
    eval_dataset = pretrain_dataset['test'],
    callbacks = [EarlyStoppingCallback(patience = 3)]
)

# 훈련 시작
trainer.train()

Step,Training Loss,Validation Loss
500,7.548800,7.209426
1000,7.199600,7.069717
1500,7.081100,6.915367
2000,6.934800,6.838158
2500,6.908200,6.733906
3000,6.756900,6.651452
3500,5.883700,6.803591
4000,4.806200,6.711576
4500,4.815900,6.654977


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


TrainOutput(global_step=4500, training_loss=6.4372587890625, metrics={'train_runtime': 3375.2853, 'train_samples_per_second': 23.346, 'train_steps_per_second': 2.919, 'total_flos': 5551703876980800.0, 'train_loss': 6.4372587890625, 'epoch': 1.3702801461632155})

In [ ]:
 #google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Pretrained_Model'

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/My Drive/Pretrained_Model/tokenizer_config.json',
 '/content/drive/My Drive/Pretrained_Model/special_tokens_map.json',
 '/content/drive/My Drive/Pretrained_Model/vocab.txt',
 '/content/drive/My Drive/Pretrained_Model/added_tokens.json',
 '/content/drive/My Drive/Pretrained_Model/tokenizer.json')

Labeled NSP 데이터셋으로 파인튜닝

In [ ]:
#훈련 데이터 불러오기
finetune_dataset_path = '/content/drive/My Drive/KcBERT_finetune_dataset.xlsx'
df_finetune = pd.read_excel(finetune_dataset_path, index_col = 0)

In [ ]:
#MLM으로 훈련된 모델 로드
#BertForNextSentencePrediction 구조에 맞게 모델 불러오기
pretrained_model_load_path = '/content/drive/My Drive/Pretrained_Model'

pretrained_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_load_path)
pretrained_model = BertForNextSentencePrediction.from_pretrained(pretrained_model_load_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at /content/drive/My Drive/Pretrained_Model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(df_finetune)
dataset = Dataset.from_pandas(df_finetune)
print(dataset)
print(df_finetune[df_finetune['sent2'].isnull()])

                                                  sent1  \
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
4     모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
...                                                 ...   
5819   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5820   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5821   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5822   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
5823   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                  sent2  label  sent2_index  
0     저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...      1          0.0  
1     헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...      1          1.0  
2     모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 ...  

In [ ]:
# NSP 데이터셋 준비(NSP 태스크에서는 stratify 적용)
def tokenize_function(examples):
    return pretrained_tokenizer(examples['sent1'], examples['sent2'], truncation = True, padding = "max_length", max_length = pretrained_model.config.max_position_embeddings)

dataset = Dataset.from_pandas(df_finetune)
finetune_dataset = dataset.map(tokenize_function, batched=True)
finetune_dataset = finetune_dataset.class_encode_column('label')
finetune_dataset = finetune_dataset.train_test_split(test_size = 0.1, stratify_by_column = 'label')

Map:   0%|          | 0/5824 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/5824 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/5824 [00:00<?, ? examples/s]

In [ ]:
#훈련 설정
training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'steps',
    eval_steps = 200,
    save_strategy = "steps",
    save_steps = 200,
    num_train_epochs = 3,
    save_total_limit = 2,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    warmup_steps = 200, #고려사항
    weight_decay = 0.01, #고려사항
    logging_dir = "./logs",
    load_best_model_at_end = True
)

# 트레이너 생성
trainer = Trainer(
    model = pretrained_model,
    args = training_args,
    train_dataset = finetune_dataset['train'],
    eval_dataset = finetune_dataset['test'],
    callbacks = [EarlyStoppingCallback(patience = 3)]
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
200,No log,0.381404
400,No log,0.514776
600,0.383900,0.344357
800,0.383900,0.415382
1000,0.321400,0.428262
1200,0.321400,0.438409


TrainOutput(global_step=1200, training_loss=0.3405350685119629, metrics={'train_runtime': 647.5672, 'train_samples_per_second': 24.28, 'train_steps_per_second': 3.039, 'total_flos': 1478920519962000.0, 'train_loss': 0.3405350685119629, 'epoch': 1.8292682926829267})

In [ ]:
#google drive에 학습된 모델을 저장
save_path = '/content/drive/My Drive/Finetuned_Model'

pretrained_model.save_pretrained(save_path)
pretrained_tokenizer.save_pretrained(save_path)

('/content/drive/My Drive/Finetuned_Model/tokenizer_config.json',
 '/content/drive/My Drive/Finetuned_Model/special_tokens_map.json',
 '/content/drive/My Drive/Finetuned_Model/vocab.txt',
 '/content/drive/My Drive/Finetuned_Model/added_tokens.json',
 '/content/drive/My Drive/Finetuned_Model/tokenizer.json')

#모델 사용

In [ ]:
#NSP로 훈련된 모델 로드
finetuned_model_load_path = '/content/drive/My Drive/Finetuned_Model'

finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_load_path)
finetuned_model = BertForNextSentencePrediction.from_pretrained(finetuned_model_load_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#카카오톡 데이터 불러오기
file_path = '/content/drive/My Drive/judge_question_result_long.csv'

In [ ]:
#카카오톡 대화내용을 데이터프레임으로 받기
df = pd.read_csv(file_path)

#질문 딕셔너리, 답변 목록 리스트(이중 리스트) 생성
questions, answer_lists = {}, []

#질문으로 판별된 텍스트를 새 데이터프레임으로 생성
df_question = df[df['label'] == 'question']

print(df_question)
#새 데이터프레임의 index, text를 question 딕셔너리에 저장
questions = {text : index for (index, text) in zip(df_question.index, df_question['text'])}
print(questions)

       Unnamed: 0.1  Unnamed: 0                 name  \
0                 0           0   하희철 / 기획, 마케팅, 노코드   
1                 1           1             (알 수 없음)   
2                 2           2   하희철 / 기획, 마케팅, 노코드   
10               10          10   하희철 / 기획, 마케팅, 노코드   
11               11          11                  한석희   
...             ...         ...                  ...   
17226         17226       17263                 코딩라멘   
17228         17228       17265          유상수/서비스기획취준   
17231         17231       17268               박서진/기획   
17233         17233       17270                   민호   
17234         17234       17271               박서진/기획   

                                                    text     label  
0                                                 안녕하세요~  question  
1                                                 안녕하세요!  question  
2              방 만든지 몇분 만에 들어오셔서 놀랐습니다 ㅎㅎ 노코드 방에서 오신건가요?  question  
10                                                안

In [ ]:
# Ensure using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = finetuned_model.to(device)
tokenizer = finetuned_tokenizer

cuda


In [ ]:
#중복 질문 제거를 동시에 수행하기 위한 새로운 알고리즘

#실험을 위한 하이퍼파라미터 설정
#후보군으로 삼을 텍스트 개수 범위
text_range = 20
#답변 목록에 추가할지 기준이 되는 레이블값
standard = 0.5

#질문-답변 쌍 딕셔너리 생성
qa_pair_dictionary = {index : {'질문' : question, '답변' : []} for index, question in zip(df_question.index, df_question['text'])}

#질문 속 질문인지 판별할 때 사용할 불리언
in_question_texts = False

all_texts = {index : item for index, item in zip(df.index, df['text'])}

for index, item in tqdm(all_texts.items(), desc = 'Processing Answer to Question'):

  candidate_qa_list = [] #현재 텍스트의 소속을 판정할 (질문-답변 딕셔너리) 리스트
  candidate_qa_index_list = []
  in_question_texts = True if df.iloc[index]['label'] == 'question' else False #판별할 텍스트가 질문인지 검사

  start = 0 if index < 20 else index - text_range - 1 #인덱스가 20 미만일 경우 검사 범위 조정
  for i in range(start, index): #현재 텍스트가 소속될 질문의 범위
    candidate = qa_pair_dictionary.get(i, None) #qa_pair_dictionary에서 i 인덱스에 해당하는 질답 딕셔너리 가져오기
    if candidate != None:
      candidate_qa_list.append(candidate) #결과 리스트에서 최대 확률인 질문을 인덱싱하기 위해 인덱스를 포함한 딕셔너리를 append
      candidate_qa_index_list.append(i)

  if len(candidate_qa_list) == 0:
    continue

  #데이터를 튜플로 묶은 뒤 배치처리
  batched_data = [(qa_dict['질문'] + ' '.join(qa_dict['답변']), item) for qa_dict in candidate_qa_list]
  inputs = tokenizer(batched_data, padding = True, truncation = True, return_tensors = 'pt')
  inputs = {k: v.to(device) for k, v in inputs.items()}

  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)

  outputs = [{'scores' : value, 'index' : index} for value, index in zip(probabilities.tolist(), candidate_qa_index_list)]

  # 'scores'의 두 번째 값(연속적인 문장일 확률)에 따라 내림차순으로 정렬
  sorted_output = sorted(outputs, key = lambda x : x['scores'][1], reverse=True)

  output = sorted_output[0] #텍스트가 소속될 질문

  if output['scores'][1] > standard:
    #모순이 발생하지 않기 위해서 질문 속 질문으로 판별된 경우 즉시 qa_pair_dictionary에서 해당 질문을 삭제해야 함.
    if in_question_texts == True:
        del qa_pair_dictionary[index]

    qa_pair_dictionary[output['index']]['답변'].append(item)


  # print('\n',qa_pair_dictionary)
  # print('\n',used_question_index)


Processing Answer to Question: 100%|██████████| 17242/17242 [09:33<00:00, 30.07it/s]


In [ ]:
print(qa_pair_dictionary)

In [ ]:
data_result= [item for item in qa_pair_dictionary.values()]

#결과 데이터를 데이터프레임으로 변환
df_result = pd.DataFrame(data = data_result, columns = ['질문', '답변'])

In [ ]:
df_result = df_result.rename(columns = {'질문' : 'question', '답변' : 'answer'})

In [ ]:
print(df_result)

                                               question  \
0                                                안녕하세요~   
1                                                안녕하세요!   
2             방 만든지 몇분 만에 들어오셔서 놀랐습니다 ㅎㅎ 노코드 방에서 오신건가요?   
3                                                안녕하세요~   
4                                         안녕하세요. 버블 전용?   
...                                                 ...   
3182                                              안녕하세요   
3183                           버블에서 성인인증을 추가할 방법이 있을까요?   
3184               │커뮤니티│모두의노코드│모두를 위한 노코드 빌딩 포럼 커뮤니티 -   
3185   안녕하세요! 버블을 써보려고 하는데요 궁금한 점이 있습니다! 버블로 구축할때 검색...   
3186   사진 안녕하세요~혹시 스케줄에 예약된 항목에 대한 정보를 버블 front단에서 가...   

                                                 answer  
0                                                    []  
1                                                    []  
2     [ 아 안녕하세요 아니요 검색해서 들어왔습니다,  저는 노코드 관련 활동을 조금 하...  
3                                                    []  
4

In [ ]:
#.xlsx 파일로 google drive에 저장
#long 파일의 경우 illegalcharactererror로 인해서 .csv 파일로 저장하기, short 파일은 오류 발생 X.
save_path = '/content/drive/My Drive/judge_answer_result_KcBERT_long.csv'

df_result.to_csv(save_path)